# Trying to make an IDM architecture 

### Step 1: Load the image dataset 

In [73]:
import torchvision 
import os
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

Loading the images from the data. 
First getting the filenames

In [74]:
data_dir = 'c:\\Users\\aniru\\Desktop\\Code\\VPTAirSim\\data'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']

dataset_size = len(filenames)
dataset = torch.zeros(dataset_size - 4, 15, 128, 128)

Then using the filenames to load images.

Here evey image is concatinated with 2 images before it and 2 images after it.

In [75]:
for i in range(dataset_size-4):
 t_minus_2 = torchvision.io.read_image(os.path.join(data_dir, filenames[i]))
 t_minus_1 = torchvision.io.read_image(os.path.join(data_dir, filenames[i+1]))
 t = torchvision.io.read_image(os.path.join(data_dir, filenames[i+2]))
 t_plus_1 = torchvision.io.read_image(os.path.join(data_dir, filenames[i+3]))
 t_plus_2 = torchvision.io.read_image(os.path.join(data_dir, filenames[i+4]))

 dataset[i] = torch.cat((t_minus_2, t_minus_1, t ,t_plus_1, t_plus_2))
 


Normalizing the pixel value by dividing it by 255. Now it is between 0 and 1

In [76]:
dataset.size()
dataset = dataset / 255.0

This is not a final model. I am writing this to make proper data shape. 

In [77]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # 3 is input channel because of RGB * 5 images. 
        # 128 is the output channel or learnable filters
        # Kernel size 15 is temporal kernel width (3 RGB channel * 5 images)
        # (1*1) is spatial kernel width
        self.conv3d = nn.Conv3d(15, 128, kernel_size=(15, 1, 1))
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv3d(x)
        x = self.relu(x)
        return x

In [78]:
model = MyModel()

This is to make 128 size chunks of frames

In [79]:
# Create a TensorDataset
data = TensorDataset(dataset)

# Create a DataLoader
dataloader = DataLoader(data, batch_size=128, shuffle=False)


For Conv3D the input format is (batch_size, num_channels, num_frames, height, width)
So I am using unsqueeze to increase the outer dimension to make batch_size = 1 . 

Then using the permute to make the dimension in proper shape.

In [80]:
# Iterate over the dataloader in batches
for framechunk in dataloader:
    # Access the batched tensor data
    # Pass the input through the model
    # print(batch[0].unsqueeze(0).size())
    output = model(framechunk[0].unsqueeze(0).permute(0, 2, 1, 3, 4) )

    print(output.shape)  # Shape of the output tensor

torch.Size([1, 128, 114, 128, 128])
torch.Size([1, 128, 114, 128, 128])
torch.Size([1, 128, 19, 128, 128])
